In [31]:
import tensorflow as tf

In [32]:
print(tf.__version__)

2.7.0


In [33]:
import requests
import pandas as pd
import numpy as np

In [34]:
modelo = tf.keras.models.load_model('rede1.h5')

In [35]:
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              1801000   
                                                                 
 dense_1 (Dense)             (None, 500)               500500    
                                                                 
 dense_2 (Dense)             (None, 1)                 501       
                                                                 
Total params: 2,302,001
Trainable params: 2,302,001
Non-trainable params: 0
_________________________________________________________________


In [42]:


url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemPorDia?diaNum=20220530"

In [43]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [44]:
df.head()

,campoX,ticker
0,"1.130710, 1.122980, 1.120169, 1.120169, 1.132...",ABEV3
1,"2.112299, 2.112299, 2.101604, 2.106952, 2.112...",AERI3
2,"1.698991, 1.657306, 1.678806, 1.668714, 1.673...",ALPA4
3,"1.033316, 1.010411, 1.023946, 1.028110, 1.034...",ALSO3
4,"1.395257, 1.367589, 1.375494, 1.375494, 1.391...",AMAR3


In [45]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [46]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(88, 1800)

In [48]:
result = modelo.predict(X)

In [49]:
type(result)

numpy.ndarray

In [54]:
df['resultado'] = result

In [57]:
df.head()

,ticker,resultado
0,ABEV3,0.160397
1,AERI3,0.239546
2,ALPA4,0.352048
3,ALSO3,0.128953
4,AMAR3,0.292051


In [56]:
df.drop('campoX', inplace=True, axis=1)

In [58]:
json = df.to_json()

In [59]:
print(json)

{"ticker":{"0":"ABEV3","1":"AERI3","2":"ALPA4","3":"ALSO3","4":"AMAR3","5":"ANIM3","6":"AZUL4","7":"B3SA3","8":"BBAS3","9":"BBDC4","10":"BBSE3","11":"BEEF3","12":"BKBR3","13":"BPAC11","14":"BPAN4","15":"BRAP4","16":"BRFS3","17":"BRKM5","18":"BRML3","19":"CCRO3","20":"CIEL3","21":"CMIG4","22":"COGN3","23":"CPFE3","24":"CPLE6","25":"CRFB3","26":"CSAN3","27":"CSMG3","28":"CSNA3","29":"CVCB3","30":"CYRE3","31":"ECOR3","32":"EGIE3","33":"ELET3","34":"ELET6","35":"EMBR3","36":"ENBR3","37":"ENEV3","38":"ENGI11","39":"EQTL3","40":"EZTC3","41":"FLRY3","42":"GGBR4","43":"GOAU4","44":"GOLL4","45":"HAPV3","46":"HYPE3","47":"ITUB4","48":"JBSS3","49":"JHSF3","50":"KLBN11","51":"LCAM3","52":"LIGT3","53":"LREN3","54":"LWSA3","55":"MGLU3","56":"MOVI3","57":"MRFG3","58":"MRVE3","59":"MULT3","60":"MYPK3","61":"NEOE3","62":"NTCO3","63":"PCAR3","64":"PETR4","65":"POSI3","66":"PRIO3","67":"PTBL3","68":"QUAL3","69":"RADL3","70":"RAIL3","71":"RAPT4","72":"RENT3","73":"SANB11","74":"SBSP3","75":"SLCE3","76":"S

In [60]:
url = 'https://www.w3schools.com/python/demopage.php'
x = requests.post(url, data = json)

## Experiencias

In [50]:
dfTicker = df.set_index('ticker')
dfTicker.head()

,campoX
ticker,
ABEV3,"1.130710, 1.122980, 1.120169, 1.120169, 1.132..."
AERI3,"2.112299, 2.112299, 2.101604, 2.106952, 2.112..."
ALPA4,"1.698991, 1.657306, 1.678806, 1.668714, 1.673..."
ALSO3,"1.033316, 1.010411, 1.023946, 1.028110, 1.034..."
AMAR3,"1.395257, 1.367589, 1.375494, 1.375494, 1.391..."


In [51]:
dfTratadoX = dfTicker.apply(trataLinhaX, axis='columns')

In [52]:
dfTratadoX.head()

ticker
ABEV3    [1.13071, 1.12298, 1.120169, 1.120169, 1.13211...
AERI3    [2.112299, 2.112299, 2.101604, 2.106952, 2.112...
ALPA4    [1.698991, 1.657306, 1.678806, 1.668714, 1.673...
ALSO3    [1.033316, 1.010411, 1.023946, 1.02811, 1.0348...
AMAR3    [1.395257, 1.367589, 1.375494, 1.375494, 1.391...
dtype: object

In [53]:
result = modelo.predict(dfTratadoX)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).